1. Racer
Extra tasks to the given tutorial:

Adding randomly appearing coins on the road
Showing the number of collected coins in the top right corner
Comment your code

In [ ]:
import pygame, sys
from pygame.locals import *
import random, time

pygame.init()
pygame.mixer.init()
coin_sound = pygame.mixer.Sound("Coin.wav")

FPS = 60
FramePerSec = pygame.time.Clock()
#дефолт параметры 
BLUE = (0, 0, 255)
RED = (255, 0, 0)
GREEN = (0, 255, 0)
BLACK = (0, 0, 0)
WHITE = (255, 255, 255)

SCREEN_WIDTH = 400
SCREEN_HEIGHT = 600
SPEED = 5
SCORE = 0
COINS = 0

font = pygame.font.SysFont("Verdana", 60)
font_small = pygame.font.SysFont("Verdana", 20)
game_over = font.render("Game over", True, BLACK)

background = pygame.image.load("AnimatedStreet.png")
background = pygame.transform.scale(background, (400, 600))

DISPLAYSURF = pygame.display.set_mode((400, 600))
DISPLAYSURF.fill(WHITE)
pygame.display.set_caption("Racer")

class Enemy(pygame.sprite.Sprite):
    def __init__(self):
        super().__init__()
        self.image = pygame.image.load("Enemy.png")
        self.rect = self.image.get_rect()
        self.rect.center = (random.randint(40, SCREEN_WIDTH - 40), 0)

    def move(self):
        global SCORE
        self.rect.move_ip(0, SPEED)
        if self.rect.top > SCREEN_HEIGHT:   
            SCORE += 1
            self.rect.top = 0
            self.rect.center = (random.randint(40, SCREEN_WIDTH - 40), 0)

class Player(pygame.sprite.Sprite):
    def __init__(self):
        super().__init__()
        self.image = pygame.image.load("Player.png")
        self.rect = self.image.get_rect()
        self.rect.center = (160, 520)

    def move(self):
        pressed_keys = pygame.key.get_pressed()
        if self.rect.left > 0 and pressed_keys[K_LEFT]:   
            self.rect.move_ip(-5, 0)
        if self.rect.right < SCREEN_WIDTH and pressed_keys[K_RIGHT]:  
            self.rect.move_ip(5, 0)

class Coin(pygame.sprite.Sprite):
    def __init__(self):
        super().__init__()
        self.image = pygame.image.load("Coin.jpg")
        self.image = pygame.transform.scale(self.image, (30, 30))  # уменшил биткоин 
        self.rect = self.image.get_rect()
        self.rect.center = (random.randint(40, SCREEN_WIDTH - 40), 0)

    def move(self):
        self.rect.move_ip(0, SPEED / 2)  # Скорость падение монеты
        if self.rect.top > SCREEN_HEIGHT: 
            self.rect.top = 0
            self.rect.center = (random.randint(40, SCREEN_WIDTH - 40), 0)

    def relocate(self):
        while True:  # Монеталарды бір-біріне қабаттастырмау үшін жаңа орын табамыз
            self.rect.center = (random.randint(40, SCREEN_WIDTH - 40), 0)
            if not any(self != coin and self.rect.colliderect(coin.rect) for coin in coins):  
                break

P1 = Player()
E1 = Enemy()

enemies = pygame.sprite.Group()
enemies.add(E1)

coins = pygame.sprite.Group()
for _ in range(3):   
    coin = Coin()
    coins.add(coin)

all_sprites = pygame.sprite.Group()
all_sprites.add(P1, E1)
all_sprites.add(*coins)

INC_SPEED = pygame.USEREVENT + 1
pygame.time.set_timer(INC_SPEED, 1000) 

while True:
    for event in pygame.event.get():
        if event.type == INC_SPEED:
            SPEED += 0.5  # Ойын барған сайын жылдамдайды
        if event.type == QUIT:
            pygame.quit()
            sys.exit()

    DISPLAYSURF.blit(background, (0, 0))
    
    scores = font_small.render(str(SCORE), True, BLACK)
    coins_text = font_small.render(f"Тиын: {COINS}", True, BLACK)
    DISPLAYSURF.blit(scores, (10, 10))
    DISPLAYSURF.blit(coins_text, (SCREEN_WIDTH - 100, 10))

    for entity in all_sprites:
        DISPLAYSURF.blit(entity.image, entity.rect)
        entity.move() 

    for coin in coins:
        if pygame.sprite.collide_rect(P1, coin):  # звук подбора биткоина 
            coin_sound.play()
            COINS += 1
            coin.relocate()   

    if pygame.sprite.spritecollideany(P1, enemies):  # если столкнуля с врагом  то издается звук столкновоние 
        pygame.mixer.Sound('crash.wav').play()
        time.sleep(0.5)
        DISPLAYSURF.fill(RED)  # Экран қызыл түске боялады
        DISPLAYSURF.blit(game_over, (30, 250))
        pygame.display.update()
        for entity in all_sprites:
            entity.kill()
        time.sleep(2)
        pygame.quit()
        sys.exit()

    pygame.display.update()
    FramePerSec.tick(FPS)   


SystemExit: 